In [ ]:
import os
import zipfile

!wget "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"

with zipfile.ZipFile("./cats_and_dogs_filtered.zip","r") as zip_ref:
    zip_ref.extractall()

base_dir = './cats_and_dogs_filtered'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

In [ ]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.nn import Conv2d, ReLU, Sigmoid, MaxPool2d

import time
import PIL
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.ion()   # interactive mode

# Batch size
bs = 20 

# Training iteration
epochs = 30

# Number of classes
num_classes = 2

device = ("cuda" if torch.cuda.is_available() else "cpu") # Use GPU or CPU for training

In [ ]:
# Data augmentation and normalization for training
# Just normalization and resizing for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize([224,224]),
        transforms.RandomRotation(5),
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(224, scale=(0.96, 1.0), ratio=(0.95, 1.05)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize([224,224]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
}

# Pass in our transforms here
train_data = datasets.ImageFolder(root=train_dir, transform=data_transforms['train'])
validation_data = datasets.ImageFolder(root=validation_dir, transform=data_transforms['val'])

# Get the size of our data, which will be used when calculating the average loss and accuracy
train_data_size = len(train_data)
valid_data_size = len(validation_data)

print(train_data_size)
print(valid_data_size)

# Used to import our dataset
trainloader = torch.utils.data.DataLoader(dataset=train_data, batch_size=bs, shuffle=True)
validationloader = torch.utils.data.DataLoader(dataset=validation_data, batch_size=bs, shuffle=True)

2000
1000


In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.convolutional_neural_network_layers = nn.Sequential(
            Conv2d(in_channels=3, out_channels=16, kernel_size=(3,3)),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2),

            Conv2d(16, 32, kernel_size=(3,3)),
            ReLU(),
            MaxPool2d(kernel_size=2),

            Conv2d(32, 64, kernel_size=(3,3)),
            ReLU(),
            MaxPool2d(kernel_size=2)
        )

        self.linear_layers = nn.Sequential(
            nn.Linear(43264, 10),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(10, 2),
            nn.Sigmoid()
        )
  
    def forward(self, inputs):
        # Pass the input tensor though each of our operations
        x = self.convolutional_neural_network_layers(inputs) # conv layer
        # Transforms the output from a convolutional layer into a dense layer
        size = x.shape[0]
        x = x.view(size, -1) # Flatten
        x = self.linear_layers(x) # Fully connected layer
        return x

In [ ]:
model = Model()
model.to(device)
print(model)

Model(
  (convolutional_neural_network_layers): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=43264, out_features=10, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=10, out_features=2, bias=True)
    (4): Sigmoid()
  )
)


In [ ]:
# Observe that all parameters are being optimized
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
  train_loss, val_loss = [], []

for epoch in range(epochs):
   
    total_train_loss = 0
    total_val_loss = 0

    model.train()
    
    # training our model
    for idx, (image, label) in enumerate(trainloader):

        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        pred = model(image)

        loss = criterion(pred, label)
        total_train_loss += loss.item()

        loss.backward()
        optimizer.step()

      total_train_loss = total_train_loss / (idx + 1)
      train_loss.append(total_train_loss)
    
    # validating our model
    model.eval()
    total = 0
    for idx, (image, label) in enumerate(validationloader):
        image, label = image.cuda(), label.cuda()
        pred = model(image)
        loss = criterion(pred, label)
        total_val_loss += loss.item()

        pred = torch.nn.functional.softmax(pred, dim=1)
        for i, p in enumerate(pred):
            if label[i] == torch.max(p.data, 0)[1]:
                total = total + 1

    accuracy = total / valid_data_size

    total_val_loss = total_val_loss / (idx + 1)
    val_loss.append(total_val_loss)

    if epoch % 5 == 0:
        print('\nEpoch: {}/{}, Train Loss: {:.4f}, Val Loss: {:.4f}, Val Acc: {:.4f}'.format(epoch + 1, epochs, total_train_loss, total_val_loss, accuracy))


Epoch: 1/30, Train Loss: 0.6928, Val Loss: 0.6931, Val Acc: 0.5000


In [ ]:
fig=plt.figure(figsize=(20, 10))
plt.plot(np.arange(1, epochs+1), train_loss, label="Train loss")
plt.plot(np.arange(1, epochs+1), val_loss, label="Validation loss")
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.title("")
plt.legend(['Training Loss', 'Validation Loss'], loc='upper right')